<a href="https://colab.research.google.com/github/Santosh-Gupta/NaturalLanguageRecommendations/blob/srihari-dev/notebooks/model_debug.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers --quiet
%tensorflow_version 2.x

TensorFlow 2.x selected.


In [2]:
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Lambda, Dense, Activation, Concatenate
from transformers import TFBertModel
print('TensorFlow:', tf.__version__)

TensorFlow: 2.1.0-rc1


In [3]:
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
    print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    strategy = tf.distribute.MirroredStrategy()

print("REPLICAS: ", strategy.num_replicas_in_sync)

INFO:absl:Entering into master device scope: /job:worker/replica:0/task:0/device:CPU:0


Running on TPU  ['10.3.172.90:8470']
INFO:tensorflow:Initializing the TPU system: 10.3.172.90:8470


INFO:tensorflow:Initializing the TPU system: 10.3.172.90:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


REPLICAS:  8


In [0]:
batch_size = 12 * strategy.num_replicas_in_sync
embedding_dim = 512
autotune = tf.data.experimental.AUTOTUNE

In [0]:
def get_random_title():
    return tf.random.uniform(shape=[512], maxval=200, dtype=tf.int32)

def get_random_citation():
    vector = tf.random.uniform(shape=[embedding_dim], minval=-1, maxval=1, dtype=tf.float32)
    normed_vector = tf.math.l2_normalize(vector)
    return normed_vector

def generate_sample():
    title = get_random_title()
    posCitations = get_random_citation()
    return (title, posCitations), tf.constant(1., dtype=tf.float32)

In [0]:
@tf.function
def loss_fn(_, probs):
    '''
        1. Every sample is its own positive, and  the rest of the
            elements in the batch are its negative.
        2. Each TPU core gets 1/8 * global_batch_size elements, hence
            compute shape dynamically.
        3. Dataset produces dummy labels to make sure the loss_fn matches
            the loss signature of keras, actual labels are computed inside this
            function.
    '''
    bs = tf.shape(probs)[0] 
    labels = tf.eye(bs, bs)
    return tf.losses.categorical_crossentropy(labels, probs)
    
def create_model():
    title = tf.keras.Input(shape=(512,), dtype=tf.int32)    # from bert tokenizer
    citation = tf.keras.Input(shape=(512,))                 # normalized word2vec outputs
    
    bert_model = TFBertModel.from_pretrained('bert-base-uncased')
    
    titleOut = bert_model(title)
    titleOutMean = tf.reduce_mean(titleOut[0], axis=1)
    titleOutSim = Dense(units=embedding_dim, activation='tanh', name='DenseTitle')(titleOutMean)

    citationSim = Dense(units=embedding_dim, activation='tanh', name='DenseCitation')(citation)

    # Get dot product of each of title x citation combinations
    dotProduct = tf.reduce_sum(tf.multiply(titleOutSim[:, None, :], citationSim), axis=-1)
    
    # Softmax to make sure each row has sum == 1.0
    probs = tf.nn.softmax(dotProduct, axis=-1)

    model = tf.keras.Model(inputs=[title, citation], outputs=[probs])
    return model

In [0]:
with strategy.scope():
    model = create_model()
    model.compile(loss=loss_fn, optimizer=tf.optimizers.Adam())

In [8]:
with strategy.scope():
    dataset = tf.data.Dataset.range(10000)
    dataset = dataset.shuffle(512)
    dataset = dataset.map(lambda _ : generate_sample(), num_parallel_calls=autotune)
    dataset = dataset.batch(batch_size, drop_remainder=False)
    dataset = dataset.prefetch(autotune)
    print(tf.data.experimental.get_structure(dataset))

((TensorSpec(shape=(None, 512), dtype=tf.int32, name=None), TensorSpec(shape=(None, 512), dtype=tf.float32, name=None)), TensorSpec(shape=(None,), dtype=tf.float32, name=None))


In [9]:
model.fit(dataset, epochs=5, steps_per_epoch=20)

Train for 20 steps
Epoch 1/5


20/20 [==============================] - 76s 4s/step - loss: 2.6067
Epoch 2/5
20/20 [==============================] - 10s 476ms/step - loss: 2.5318
Epoch 3/5
20/20 [==============================] - 10s 476ms/step - loss: 2.5017
Epoch 4/5
20/20 [==============================] - 9s 475ms/step - loss: 2.4914
Epoch 5/5
20/20 [==============================] - 10s 480ms/step - loss: 2.4877
